# Winning Jeopardy
*Jeopardy!* is a long-running game show where participants answer questions to win money. If you want to compete on Jeopardy, and you're looking for any competitive edge to help you win, then you should work with a dataset of *Jeopardy!* questions to figure out some patterns in the questions that could help you win. This is exactly what one *Jeopardy!* winner did, [here](https://vimeo.com/29001512) is a video of him breaking down his strategy.

## The Dataset
The dataset is named `jeopardy.csv`, and contains 20000 rows from the beginning of a full dataset of Jeopardy questions, which you can download [here](https://www.reddit.com/r/datasets/comments/1uyd0t/200000_jeopardy_questions_in_a_json_file).

Let's preview the dataset:

In [1]:
import pandas as pd
jeopardy = pd.read_csv("jeopardy.csv")
jeopardy.head(5)

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


Each row in the dataset represents a single question on a single episode of *Jeopardy!*. Here are explanations of each column:
 
| Column | Description  |
| :-----: | :----------- :
|Show Number | The Jeopardy episode number of the show this question was in |
| Air Date | The date the episode aired |
| Round | The round of Jeopardy that the question was asked in. Jeopardy has several rounds as each episode progresses |
| Category | The category of the question |
| Value | The number of dollars answering the question correctly is worth |
| Question | The text of the question |
| Answer | The text of the answer |

Now that the data is imported and we understand what each column represents, let's tidy it up!

In [2]:
jeopardy.columns

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')

In [3]:
jeopardy.columns = ['Show Number', 'Air Date', 'Round', 'Category', 'Value', 'Question', 'Answer']

## Normalizing Text
Before we can start doing analysis on the *Jeopardy!* questions, we need to normalize all of the text columns (the `Question` and `Answer` columns). The idea behind normalization is to ensure that all words are lowercase and punctuation is removed so `Don't` and `don't` aren't considered to be different words when they're compared.

The `Value` column should also be numeric, allowing it to be manipulated it more easily. We need to remove the dollar sign from the beginning of each value and convert the column from text to numeric.

In [4]:
import re

def normalize_text(text):
    text = text.lower()
    text = re.sub("[^A-Za-z0-9\s]", "", text)
    return text

def normalize_values(text):
    text = re.sub("[^A-Za-z0-9\s]", "", text)
    try:
        text = int(text)
    except Exception:
        text = 0
    return text

In [5]:
jeopardy["clean_question"] = jeopardy["Question"].apply(normalize_text)
jeopardy["clean_answer"] = jeopardy["Answer"].apply(normalize_text)
jeopardy["clean_value"] = jeopardy["Value"].apply(normalize_values)

After normalizing the those columns, let's take another look at the data.

In [6]:
jeopardy.head(5)

,Show Number,Air Date,Round,Category,Value,Question,Answer,clean_question,clean_answer,clean_value
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus,for the last 8 years of his life galileo was u...,copernicus,200
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe,no 2 1912 olympian football star at carlisle i...,jim thorpe,200
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona,the city of yuma in this state has a record av...,arizona,200
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's,in 1963 live on the art linkletter show this c...,mcdonalds,200
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams,signer of the dec of indep framer of the const...,john adams,200


One last adjustment: The `Air Date` column should also be a datetime, not a string, to enable us to work with it more easily.

In [7]:
jeopardy["Air Date"] = pd.to_datetime(jeopardy["Air Date"])

In [8]:
jeopardy.dtypes

Show Number                int64
Air Date          datetime64[ns]
Round                     object
Category                  object
Value                     object
Question                  object
Answer                    object
clean_question            object
clean_answer              object
clean_value                int64
dtype: object

## Answers in the Questions
In order to figure out whether to study past questions, study general knowledge, or not study it all, it would be helpful to figure out two things:

- How often the answer is deducible from the question.
- How often new questions are repeats of older questions.

We can answer the second question by seeing how often complex words (> 6 characters) reoccur. We can answer the first question by seeing how many times words in the answer also occur in the question. We'll work on the first question now, and come back to the second.

In [9]:
def count_matches(row):
    split_answer = row["clean_answer"].split(" ")
    split_question = row["clean_question"].split(" ")
    if "the" in split_answer:
        split_answer.remove("the")
    if len(split_answer) == 0:
        return 0
    match_count = 0
    for item in split_answer:
        if item in split_question:
            match_count += 1
    return match_count / len(split_answer)

jeopardy["answer_in_question"] = jeopardy.apply(count_matches, axis=1)

In [10]:
jeopardy["answer_in_question"].mean()

0.059357587183968614

## Findings
The answer only appears in the question about 6% of the time. This isn't a huge number, and means that we probably can't just hope that hearing a question will enable us to figure out the answer. We'll probably have to study.

## Recycled Questions
Let's say we want to investigate how often new questions are repeats of older ones, here's one approach we can take:
- Sort `jeopardy` in order of ascending air date
- Maintain a set called `terms_used` that will be empty initially
- Iterate through each row of `jeopardy`
- Split `clean_question` into words, remove any word shorter than `6` characters, and check if each word occurs in `terms_used`
 - If it does, increment a counter
 - Add each word to `terms_used`
  
This will enable us to check if the terms in questions have been used previously or not. Only looking at words greater than `6` characters enables you to filter out words `like` the and `than`, which are commonly used, but don't tell you a lot about a question.

In [11]:
jeopardy = jeopardy.sort_values("Air Date")
question_overlap = []
terms_used = set()
for i, row in jeopardy.iterrows():
        split_question = row["clean_question"].split(" ")
        split_question = [q for q in split_question if len(q) > 5]
        match_count = 0
        for word in split_question:
            if word in terms_used:
                match_count += 1
        for word in split_question:
            terms_used.add(word)
        if len(split_question) > 0:
            match_count /= len(split_question)
        question_overlap.append(match_count)
jeopardy["question_overlap"] = question_overlap

jeopardy["question_overlap"].mean()

0.8722160146580127

## Question Overlap Observations
There is about 87% overlap between terms in new questions and terms in old questions, but our discovery method is not without problems. This only looks at a small set of questions, and it doesn't look at phrases, it looks at single terms. This makes it relatively insignificant, but it does mean that it's worth looking more into the recycling of questions.

## Low Value vs. High Value Questions
Let's say we only want to study questions that pertain to high value questions instead of low value questions. This will us you earn more money on the show!

You can actually figure out which terms correspond to high-value questions using a chi-squared test. You'll first need to narrow down the questions into two categories:

- Low value -- Any row where `Value` is less than `800`
- High value -- Any row where `Value` is greater than `800`

You'll then be able to loop through each of the terms in `terms_used`, and:

- Find the number of low value questions the word occurs in
- Find the number of high value questions the word occurs in
- Find the percentage of questions the word occurs in

Based on the percentage of questions the word occurs in, find expected counts. Compute the chi squared value based on the expected counts and the observed counts for high and low value questions. 

We can then find the words with the biggest differences in usage between high and low value questions, by selecting the words with the highest associated chi-squared values.

In [12]:
def determine_value(row):
    value = 0
    if row["clean_value"] > 800:
        value = 1
    return value

jeopardy["high_value"] = jeopardy.apply(determine_value, axis=1)

In [13]:
def count_usage(term):
    low_count = 0
    high_count = 0
    for i, row in jeopardy.iterrows():
        if term in row["clean_question"].split(" "):
            if row["high_value"] == 1:
                high_count += 1
            else:
                low_count += 1
    return high_count, low_count

comparison_terms = list(terms_used)[:5]
observed_expected = []
for term in comparison_terms:
    observed_expected.append(count_usage(term))

observed_expected

[(11, 30), (0, 2), (1, 0), (1, 2), (0, 1)]

## Applying the Chi-Squared Test
Let's use the `scipy.stats.chisquare` function to compute the chi-squared value and p-value given the expected and observed counts.

In [14]:
from scipy.stats import chisquare
import numpy as np

high_value_count = jeopardy[jeopardy["high_value"] == 1].shape[0]
low_value_count = jeopardy[jeopardy["high_value"] == 0].shape[0]

chi_squared = []
for obs in observed_expected:
    total = sum(obs)
    total_prop = total / jeopardy.shape[0]
    high_value_exp = total_prop * high_value_count
    low_value_exp = total_prop * low_value_count
    
    observed = np.array([obs[0], obs[1]])
    expected = np.array([high_value_exp, low_value_exp])
    chi_squared.append(chisquare(observed, expected))

chi_squared

[Power_divergenceResult(statistic=0.044541102507489314, pvalue=0.8328501042245996),
 Power_divergenceResult(statistic=0.78995292846670262, pvalue=0.37411435927449888),
 Power_divergenceResult(statistic=2.5317964247338085, pvalue=0.11157312838169751),
 Power_divergenceResult(statistic=0.037234093889071389, pvalue=0.846989214486915),
 Power_divergenceResult(statistic=0.39497646423335131, pvalue=0.52969509124866954)]

## Chi-Squared results
None of the terms had a significant difference in usage between high value and low value rows. Additionally, the frequencies were all lower than 5, so the chi-squared test isn't as valid. It would be better to run this test with only terms that have higher frequencies.